<a href="https://colab.research.google.com/github/Aryan-Kamboj11/Multimodal-Hate-Speech-Detection/blob/main/MultiModalDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os
import re
import pandas as pd
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from torch.utils.data import Dataset
import ast
import torch

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [3]:
clean_data = pd.read_csv("/content/drive/MyDrive/Dataset/data_transformed.csv")

In [4]:
clean_data.head()

,Unnamed: 0,text,label,image_path,cleaned_text,input_ids,attention_mask,transformed_image_path
0,0,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,"[1, 0, 0]",Dataset/images/0.jpg,rt xxsugvngxx: i ran into this holy nigga today,"tensor([ 101, 19387, 22038, 6342, 2290, 160...","tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",/content/drive/MyDrive/Dataset/transformed_ima...
1,1,“EVERYbody calling you Nigger now!” https://t....,"[1, 0, 1]",Dataset/images/1.jpg,“everybody calling you nigger now!”,"tensor([ 101, 1523, 7955, 4214, 2017, 91...","tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",/content/drive/MyDrive/Dataset/transformed_ima...
2,2,@WhiteHouse @realDonaldTrump Fuck ice. White s...,"[5, 1, 1]",Dataset/images/2.jpg,[user] [user] fuck ice. white supremacist tras...,"tensor([ 101, 1030, 2317, 4580, 1030, 26...","tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",/content/drive/MyDrive/Dataset/transformed_ima...
3,3,#sissy faggot https://t.co/bm1nk8HcYO,"[3, 3, 0]",Dataset/images/3.jpg,sissy faggot,"tensor([ 101, 1001, 24761, 6508, 6904, 138...","tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",/content/drive/MyDrive/Dataset/transformed_ima...
4,4,@Gloriko_ Nigga what? https://t.co/nOwIJtgtU1,"[4, 0, 3]",Dataset/images/4.jpg,[user] nigga what?,"tensor([ 101, 1030, 1043, 10626, 12676, 10...","tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",/content/drive/MyDrive/Dataset/transformed_ima...


In [7]:
os.makedirs("/content/drive/MyDrive/Dataset/input_ids",exist_ok = True)
os.makedirs("/content/drive/MyDrive/Dataset/attention_mask",exist_ok = True)

In [10]:
for idx,row in clean_data.iterrows():
  input = tokenizer(
      row["text"],
      padding = "max_length",
      truncation = True,
      max_length = 512,
      return_tensors = "pt"
  )
  input_ids = input["input_ids"].squeeze(0)
  attention_mask = input["attention_mask"].squeeze(0)
  torch.save(input_ids,f"/content/drive/MyDrive/Dataset/input_ids/{idx}.pt")
  torch.save(attention_mask,f"/content/drive/MyDrive/Dataset/attention_mask/{idx}.pt")
  clean_data.at[idx,"input_ids"] = f"/content/drive/MyDrive/Dataset/input_ids/{idx}.pt"
  clean_data.at[idx,"attention_mask"] = f"/content/drive/MyDrive/Dataset/attention_mask/{idx}.pt"

In [13]:
def parseLabels(label_value):
  if isinstance(label_value, str):
        cleaned = re.sub(r"[^0-9]", " ", label_value)
        parts = [int(x) for x in cleaned.split() if x.strip()]
  elif isinstance(label_value, list):
      parts = label_value
  else:
      parts = []

    # Convert to multi-hot vector
  multi_hot = [0] * 6
  for label in parts:
      if 0 <= label <= 5:
          multi_hot[label] = 1
  return multi_hot

In [16]:
clean_data["labels"] = clean_data["label"].apply(parseLabels)
print(clean_data["labels"].head())

0    [1, 1, 0, 0, 0, 0]
1    [1, 1, 0, 0, 0, 0]
2    [0, 1, 0, 0, 0, 1]
3    [1, 0, 0, 1, 0, 0]
4    [1, 0, 0, 1, 1, 0]
Name: labels, dtype: object


In [18]:
clean_data.to_csv("/content/drive/MyDrive/Dataset/data_transformed.csv",index=False)

In [19]:
clean_data["transformed_image_path"] = clean_data["transformed_image_path"].str.replace("\\", "/")

In [20]:
class MultiModalDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]

        # Load pre-saved tensors from files
        input_ids = torch.load(os.path.join("dataset", row["input_ids"])).squeeze(0)
        attention_mask = torch.load(os.path.join("dataset", row["attention_mask"])).squeeze(0)

        # Load image
        image = torch.load(row["transformed_image_path"])

        # Labels
        labels = torch.tensor(row["labels"], dtype=torch.float)

        return input_ids, attention_mask, image, labels

In [21]:
dataset = MultiModalDataset(clean_data,tokenizer)

In [22]:
dataloader = DataLoader(dataset,batch_size=32,shuffle=True)


for batch in dataloader:
  input_ids,attention_mask,image,labels = batch
  print(input_ids.shape)
  print(attention_mask.shape)
  print(image.shape)
  print(labels.shape)
  break

torch.Size([32, 512])
torch.Size([32, 512])
torch.Size([32, 3, 224, 224])
torch.Size([32, 6])


In [25]:
print(pd.read_csv("/content/drive/MyDrive/Dataset/data_transformed.csv")[["input_ids", "attention_mask"]].head())

                                       input_ids  \
0  /content/drive/MyDrive/Dataset/input_ids/0.pt   
1  /content/drive/MyDrive/Dataset/input_ids/1.pt   
2  /content/drive/MyDrive/Dataset/input_ids/2.pt   
3  /content/drive/MyDrive/Dataset/input_ids/3.pt   
4  /content/drive/MyDrive/Dataset/input_ids/4.pt   

                                      attention_mask  
0  /content/drive/MyDrive/Dataset/attention_mask/...  
1  /content/drive/MyDrive/Dataset/attention_mask/...  
2  /content/drive/MyDrive/Dataset/attention_mask/...  
3  /content/drive/MyDrive/Dataset/attention_mask/...  
4  /content/drive/MyDrive/Dataset/attention_mask/...  
